In [171]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
import patsy

file_path_filled = "../dataset/movie_metadata_cleaned_filled_values.csv"
file_path_empty  = "../dataset/movie_metadata_cleaned_empty.csv"
dta = pd.read_csv(file_path_filled)
#dta_e = pd.read_csv(file_path_empty)
pd.options.display.max_colwidth = 50
pd.options.display.max_columns = 100

str_list = ['color', 'movie_title', 'director_name', 'actor_2_name', 'genres',
            'actor_1_name', 'actor_3_name', 'plot_keywords', 'language', 'country', 'content_rating']

def removeWh(val,extra = ""):
    if type(val) == str:   
        val = val.replace("|", " ").replace("-", " ").replace("-", " ").replace("'", " ").replace(".", " ").replace(" ", "").replace(" ", "").replace(" ", "").replace(" ", "").replace(" ", "")
        if extra != "": val = extra + "_" + val
    return val

def removeStrip(val):
    if type(val) == str: 
        val = val.replace("|", " ").replace("-", "").replace("-", "").replace("'", "").replace(".", "")
    return val

def genString(val, col):
    if type(val) == str: 
        val = ' ' + val
    else:
        val = ''
    return val
#prepare data
for colname, colvalue in dta.iteritems():
    if type(colvalue[1]) == str and colname != 'movie_imdb_link' and colname != 'release_date':
        if colname == 'director_name' or colname == 'actor_1_name' or colname == 'actor_2_name' or colname == 'actor_3_name':
            dta[colname] = dta[colname].apply(lambda val: removeWh(val))
        if colname == 'genres' or colname == 'plot_keywords':
            dta[colname] = dta[colname].apply(lambda val: removeStrip(val))
        if colname == 'country':
            dta[colname] = dta[colname].apply(lambda val: removeWh(val, "country"))
        if colname == 'language':
            dta[colname] = dta[colname].apply(lambda val: removeWh(val, "language"))
        if colname == 'color':
            dta[colname] = dta[colname].apply(lambda val: removeWh(val, "color"))
        if colname == 'content_rating':
            dta[colname] = dta[colname].apply(lambda val: removeWh(val, "raiting"))
            
#generate common string
dta['string'] = ''
dta['names'] = ''
dta['category'] = ''
for colname, colvalue in dta.iteritems():
    if type(colvalue[1]) == str and colname != 'movie_imdb_link' and colname != 'release_date' and colname != 'string' and colname !='category' and colname !='names': 
        if colname == 'country' or colname == 'content_rating' or colname == 'language' or colname == 'color' :
            dta['category'] += dta[colname].apply(lambda val: genString(val, colname))
        else:
            dta['string'] += dta[colname].apply(lambda val: genString(val, colname))
        if colname == 'director_name' or colname ==  'actor_1_name' or colname == 'actor_2_name' or colname ==  'actor_3_name':
            dta['names'] += dta[colname].apply(lambda val: genString(val, colname))
            
pd.options.display.max_colwidth = 500
pd.DataFrame(dta['names']) 

,names
0,JamesCameron JoelDavidMoore CCHPounder WesStudi
1,JamesCameron KateWinslet LeonardoDiCaprio GloriaStuart
2,ColinTrevorrow BryceDallasHoward ChrisPratt LaurenLapkus
3,JamesWan PaulWalker JasonStatham VinDiesel
4,JossWhedon RobertDowneyJr ChrisHemsworth ScarlettJohansson
5,MattBirch DaveLegeno RupertGrint RalphIneson
6,ChrisBuck MauriceLaMarche JoshGad LivvyStubenrauch
7,ShaneBlack DonCheadle RobertDowneyJr StephanieSzostak
8,KyleBalda JonHamm SteveCarell SteveCoogan
9,AnthonyRusso ScarlettJohansson RobertDowneyJr ChrisEvans


In [172]:

tf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=3, stop_words='english', analyzer = 'word')
X_tf = tf_vectorizer.fit_transform(dta['string'])
tf_fr = pd.DataFrame(X_tf.todense(), columns = tf_vectorizer.get_feature_names())

################################
#names_vectorizer = TfidfVectorizer(max_df=0.9, min_df=8, use_idf = True, stop_words='english',  analyzer = 'word')
names_vectorizer = CountVectorizer(max_df=0.9,min_df=14, stop_words='english' , analyzer = 'word')
X_nam = names_vectorizer.fit_transform(dta['names'])
nam_fr = pd.DataFrame(X_nam.todense(), columns = names_vectorizer.get_feature_names())

################################
cat_vectorizer = TfidfVectorizer(max_df=1.0, min_df=4, use_idf = False, stop_words='english',  analyzer = 'word')
#cat_vectorizer = CountVectorizer(max_df=1.0,min_df=2, stop_words='english' , analyzer = 'word')
X_cat = cat_vectorizer.fit_transform(dta['category'])
cat_fr = pd.DataFrame(X_cat.todense(), columns = cat_vectorizer.get_feature_names())


strings = []
for colname, colvalue in dta.iteritems():
    if type(colvalue[1]) == str:
        strings.append(colname)
#remove old gross, budget and aspect_ratio
strings.append('gross')
strings.append('budget')
strings.append('aspect_ratio')
#remove values which 
strings.append('imdb_score')
strings.append('movie_facebook_likes')
strings.append('num_critic_for_reviews')
strings.append('num_user_for_reviews')
strings.append('num_voted_users')
num_list = dta.columns.difference(strings)
##remove strings values from original data set
dta_clean = dta[num_list]



##################
print("dta shape:")
print(dta_clean.shape)
print("tf shape:")
print(tf_fr.shape)
print("names len")
print(nam_fr.shape)
print("cat len")
print(cat_fr.shape)

#result = pd.concat([dta_clean, cat_fr, tf_fr], axis=1)
result = pd.concat([dta_clean, nam_fr, cat_fr], axis=1)
result.dropna()

print("res shape:")
print (result.shape)

result.count()

dta shape:
(4812, 11)
tf shape:
(4812, 4719)
names len
(4812, 125)
cat len
(4812, 48)
res shape:
(4812, 184)


actor_1_facebook_likes       4812
actor_2_facebook_likes       4799
actor_3_facebook_likes       4812
blockbuster_month            4812
cast_total_facebook_likes    4812
director_facebook_likes      4812
dump_month                   4812
duration                     4812
production_budget            4812
title_year                   4739
worldwide_gross              4812
adamsandler                  4812
adamscott                    4812
alpacino                     4812
angelinajolie                4812
annakendrick                 4812
annehathaway                 4812
anthonyhopkins               4812
anthonymackie                4812
antonyelchin                 4812
billcobbs                    4812
billmurray                   4812
bobhoskins                   4812
bradleycooper                4812
bradpitt                     4812
briandepalma                 4812
brucegreenwood               4812
brucemcgill                  4812
brucewillis                  4812
catherineohara

In [173]:
result.to_csv(path_or_buf="../dataset/no_imdb_names-count_cat-tf_184f.csv")

In [136]:
_y, _X = patsy.dmatrices('worldwide_gross ~  -1 + color + country + language  + content_rating',
                 dta, return_type="dataframe")

In [4]:
print(*result.columns)

actor_1_facebook_likes actor_2_facebook_likes actor_3_facebook_likes blockbuster_month cast_total_facebook_likes director_facebook_likes dump_month duration imdb_score movie_facebook_likes num_critic_for_reviews num_user_for_reviews num_voted_users production_budget title_year worldwide_gross color_blackandwhite color_color country_afghanistan country_argentina country_aruba country_australia country_austria country_bahamas country_belgium country_brazil country_bulgaria country_canada country_chile country_china country_colombia country_czechrepublic country_denmark country_estonia country_finland country_france country_georgia country_germany country_greece country_hongkong country_hungary country_iceland country_india country_iran country_ireland country_israel country_italy country_japan country_kenya country_libya country_mexico country_netherlands country_newline country_newzealand country_nigeria country_norway country_officialsite country_pakistan country_panama country_peru co

In [148]:
result

,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes,blockbuster_month,cast_total_facebook_likes,director_facebook_likes,dump_month,duration,production_budget,title_year,worldwide_gross,_n,_rez,aaronpaul,aaronyoo,adamgoldberg,adamsandler,adamscott,adamshankman,aidanquinn,alanrickman,albertbrooks,albertfinney,alexanderskarsg,alexapenavega,alfredhitchcock,aliciawitt,alpacino,alysonhannigan,andrewgarfield,angelinajolie,anglee,anjelicahuston,annakendrick,annehathaway,anneheche,anthonyhopkins,anthonymackie,antonyelchin,barrylevinson,benedictcumberbatch,benmendelsohn,bethgrant,beverlydangelo,billcobbs,billduke,billmurray,billycrudup,blythedanner,bobbalaban,...,country_netherlands,country_newzealand,country_norway,country_poland,country_romania,country_russia,country_southafrica,country_southkorea,country_spain,country_sweden,country_switzerland,country_thailand,country_uk,country_usa,language_aboriginal,language_arabic,language_cantonese,language_dari,language_dutch,language_english,language_french,language_german,language_hebrew,language_hindi,language_italian,language_japanese,language_korean,language_mandarin,language_portuguese,language_russian,language_spanish,language_usa,language_vietnamese,language_zulu,raiting_approved,raiting_g,raiting_gp,raiting_m,raiting_nc17,raiting_notrated,raiting_passed,raiting_pg,raiting_pg13,raiting_r,raiting_tv14,raiting_tvg,raiting_tvma,raiting_tvpg,raiting_unrated,raiting_x
0,1000,942.0,862,1,4859,7000,0,178,425000000,2009.0,2783918982,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.50000,0.0,0.0,0.0,0.0,0.0,0.50000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,29000,14000.0,787,1,45239,7000,0,194,200000000,1997.0,2207615668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.50000,0.0,0.0,0.0,0.0,0.0,0.50000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3000,3000.0,878,1,8694,367,0,124,215000000,2015.0,1671640593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.50000,0.0,0.0,0.0,0.0,0.0,0.50000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26000,23000.0,14000,0,79155,0,0,140,190000000,2015.0,1516748684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.50000,0.0,0.0,0.0,0.0,0.0,0.50000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,26000,21000.0,19000,0,97000,0,0,141,250000000,2015.0,1404705868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.50000,0.0,0.0,0.0,0.0,0.0,0.50000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,10000,577.0,175,1,11064,0,0,130,125000000,2011.0,1341511219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50000,0.00000,0.0,0.0,0

In [174]:
for ind, res in enumerate(result[1:2].values): #20:21
    for in_, re in enumerate(res):
        if re != 0: 
            print(result.columns[in_])
            print(re)

actor_1_facebook_likes
29000.0
actor_2_facebook_likes
14000.0
actor_3_facebook_likes
787.0
blockbuster_month
1.0
cast_total_facebook_likes
45239.0
director_facebook_likes
7000.0
duration
194.0
production_budget
200000000.0
title_year
1997.0
worldwide_gross
2207615668.0
katewinslet
1.0
leonardodicaprio
1.0
color_color
0.5
country_usa
0.5
language_english
0.5
raiting_pg13
0.5


In [134]:
print(result.columns)

Index(['actor_1_facebook_likes', 'actor_2_facebook_likes',
       'actor_3_facebook_likes', 'blockbuster_month',
       'cast_total_facebook_likes', 'director_facebook_likes', 'dump_month',
       'duration', 'production_budget', 'title_year',
       ...
       'raiting_passed', 'raiting_pg', 'raiting_pg13', 'raiting_r',
       'raiting_tv14', 'raiting_tvg', 'raiting_tvma', 'raiting_tvpg',
       'raiting_unrated', 'raiting_x'],
      dtype='object', length=490)
